In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool,CodeInterpreterTool

In [4]:
import os

In [5]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [6]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="o1",
    api_version="2025-03-01-preview",
    model = "o1",
    azure_endpoint="https://aq-aoai-swedencentral.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)

In [7]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='eastus2.api.azureml.ms;7a28b21e-0d3e-4435-a686-d92889d4ee96;AI-FOUNDRY;o-models-project',
)

In [8]:
bing_connection = project_client.connections.get(
    connection_name='aqbinggrounding'
)

conn_id = bing_connection.id

In [9]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    # with project_client:
    agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="my-web-agent",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
    )
    print(f"SMS: {message}")
        # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

        # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])

        # project_client.close()

    return messages["data"][0]["content"][0]["text"]["value"]



In [10]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="my-coder-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
        # In the save_blog_agent function, update to:
    last_message_text = None
    for msg in reversed(messages["data"]):
            if msg.get("role") == "assistant":
                last_message_text = msg["content"][0]["text"]["value"]
                break
    if last_message_text:
            print(f"Last Message: {last_message_text}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


        # project_client.close()


    return "Saved"


    
    

In [11]:
bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)

In [12]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [13]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [14]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [15]:

reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [16]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing a blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                               

    """)
)  # Stream the messages to the console.

---------- user ----------


                    I am writing a blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link

                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning


    
This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_HXIgqVAwF47wWfKz2O3Q2ZYF
Created thread, ID: thread_WXpN81FM3xCRK9ikdyfkZGmo
SMS: {'id': 'msg_GeIhNQHpG96nmAcYkaz8xX5I', 'object': 'thread.message', 'created_at': 1745161693, 'assistant_id': None, 'thread_id': 'thread_WXpN81FM3xCRK9ikdyfkZGmo', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Machine Learning? The difference between AI and ML. The history of Machine Learning. Provide references.', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStatus.COMPLE

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing a blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link\n\n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=133, completion_tokens=686), metadata={}, content=[FunctionCall(id='call_t04BxyVwlTgBH9QFzgG65omW', arguments='{"query":"What is Machine Learning? The difference between AI and ML. The history of Machine Learning. Provide references."}', name='web_ai_agent')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='bing_search_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='You can find